# Exogenous Variables for GRACE 5y forecast (Climate, Soil and Vegetation)

In this notebook we download climate variables using the CDS API, clip the data to predefined basin regions, calculate time series features, and merge the results with the existing training dataset. It leverages Dask for parallel processing, improving efficiency when working with large datasets.

In [18]:
import pandas as pd
import geopandas as gpd

# CDS API
import cdsapi

# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()

import dask
import gc
import os
import rioxarray
from shapely import wkt

In [19]:
# Instruction to show all rows and columns when printing dataframes 
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

In [65]:
URL = 'https://cds-beta.climate.copernicus.eu/api'
KEY = '43adcbfc-8934-4d43-9532-8de728a836e7'

In [24]:
# List of climate variables to download and process
variables = [
    '100m_u_component_of_wind', '100m_v_component_of_wind', '10m_u_component_of_wind', '10m_v_component_of_wind', 
    '10m_wind_speed', '2m_dewpoint_temperature', '2m_temperature', 'angle_of_sub_gridscale_orography', 
    'boundary_layer_height', 'convective_precipitation', 'convective_rain_rate', 'convective_snowfall', 
    'convective_snowfall_rate_water_equivalent', 'evaporation', 'forecast_albedo', 'forecast_surface_roughness', 
    'geopotential', 'gravity_wave_dissipation', 'high_cloud_cover', 'high_vegetation_cover', 
    'instantaneous_large_scale_surface_precipitation_fraction', 'instantaneous_moisture_flux', 
    'large_scale_precipitation', 'large_scale_rain_rate', 'large_scale_snowfall', 
    'large_scale_snowfall_rate_water_equivalent', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation', 
    'low_cloud_cover', 'low_vegetation_cover', 'mean_sea_level_pressure', 'northward_turbulent_surface_stress', 
    'potential_evaporation', 'precipitation_type', 'runoff', 'sea_surface_temperature', 'skin_reservoir_content', 
    'skin_temperature', 'slope_of_sub_gridscale_orography', 'snow_albedo', 'snow_density', 'snow_depth', 
    'snow_evaporation', 'snowfall', 'snowmelt', 'soil_temperature_level_1', 'soil_temperature_level_2', 
    'soil_temperature_level_3', 'soil_temperature_level_4', 'soil_type', 'sub_surface_runoff', 'surface_pressure', 
    'surface_runoff', 'temperature_of_snow_layer', 'total_cloud_cover', 'total_column_cloud_ice_water', 
    'total_column_cloud_liquid_water', 'total_column_water', 'total_column_water_vapour', 'total_precipitation', 
    'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 
    'volumetric_soil_water_layer_4', 'zero_degree_level', 'vertical_integral_of_northward_cloud_frozen_water_flux', 
    'vertical_integral_of_eastward_cloud_frozen_water_flux', 'vertical_integral_of_northward_cloud_liquid_water_flux'
]

# Initialize the CDS API client
c = cdsapi.Client()

# Function to download and process climate data
def download_and_process(variable):
    """
    Downloads and processes data for a given climate variable, clipping it to basin geometries and 
    calculating time series features for each basin.
    
    Parameters:
    - variable (str): The climate variable to download and process
    """
    
    # 1. Check if the file exists
    filename = f"download_{variable}.nc"
    if not os.path.exists(filename):
        # If the file doesn't exist, download the data
        print(f"Downloading {filename}...")
        c.retrieve(
            'reanalysis-era5-single-levels-monthly-means',
            {
                'format': 'netcdf',
                'product_type': 'monthly_averaged_reanalysis',
                'variable': [variable],
                'year': [str(y) for y in range(2000, 2025)],  # Data from 2000 to 2024
                'month': ['{:02d}'.format(m) for m in range(1, 13)],
                'time': '00:00',
                'area': [90, -180, -90, 180],  # Global coverage
            },
            filename
        )
    else:
        print(f"File {filename} already exists, skipping download.")
    
    # 2. Process the downloaded data
    exog_var = xr.open_dataset(filename)
    
    # Select the last available variable in the dataset
    last_variable = list(exog_var.data_vars.keys())[-1]
    print(f"Using variable '{last_variable}' from {filename}")

    # Ensure correct CRS and chunk by lat/lon
    exog_var = exog_var.rio.write_crs("EPSG:4326")
    exog_var = exog_var.chunk({'latitude': 200, 'longitude': 200, 'date': 100})

    # 3. Initialize a list for Dask tasks
    tasks = []
    
    # 4. Function to process each basin and clip exog_var
    @dask.delayed
    def process_basin(geom):
        # Clip the exog_var dataset using the basin geometry
        mask = exog_var.rio.clip([geom], exog_var.rio.crs, drop=True)
        
        # Compute the mean of the variable over the basin for each time point
        basin_series = mask[last_variable].mean(dim=['latitude', 'longitude']).compute()
        return basin_series
    
    # 5. Loop through basin geometries and apply the processing function
    for geom in gdf['geometry']:
        task = process_basin(geom)  # Create the delayed task
        tasks.append(task)  # Append it to the list
    
    # 6. Compute all tasks in parallel
    basin_time_series = dask.compute(*tasks)

    # 7. Convert the results into a DataFrame
    basin_time_series_df = pd.DataFrame({
        'basin_id': gdf['basin_id'],
        f'{last_variable}_series': [basin_time_series[i].values for i in range(len(basin_time_series))]
    })
    
    # 8. Create a new DataFrame with time series data for each basin
    exog_series_df = pd.DataFrame(basin_time_series_df[f'{last_variable}_series'].tolist()).transpose()
    exog_series_df.columns = basin_time_series_df['basin_id']
    
    # Add date, year, and month columns
    start_date = '2000-01-01'
    exog_series_df['datetime'] = pd.date_range(start=start_date, periods=exog_series_df.shape[0], freq='MS')
    exog_series_df['year'] = exog_series_df['datetime'].dt.year
    exog_series_df['month'] = exog_series_df['datetime'].dt.month

    # Calculate time series features for the climate variable
    def calculate_features(df):
        result = pd.DataFrame(index=df.index)
        result[f'{variable}_past_historic_mean'] = df.expanding().mean().shift(1)
        result[f'{variable}_past_historic_max'] = df.expanding().max().shift(1)
        result[f'{variable}_past_historic_min'] = df.expanding().min().shift(1)
        result[f'{variable}_mean_past_year'] = df.rolling(window=12, min_periods=1).mean().shift(1)
        result[f'{variable}_max_past_year'] = df.rolling(window=12, min_periods=1).max().shift(1)
        result[f'{variable}_min_past_year'] = df.rolling(window=12, min_periods=1).min().shift(1)
        result[f'{variable}_mean_past_quarter'] = df.rolling(window=3, min_periods=1).mean().shift(1)
        result[f'{variable}_max_past_quarter'] = df.rolling(window=3, min_periods=1).max().shift(1)
        result[f'{variable}_min_past_quarter'] = df.rolling(window=3, min_periods=1).min().shift(1)
        result[f'{variable}_mean_past_5years'] = df.rolling(window=60, min_periods=1).mean().shift(1)
        result[f'{variable}_max_past_5years'] = df.rolling(window=60, min_periods=1).max().shift(1)
        result[f'{variable}_min_past_5years'] = df.rolling(window=60, min_periods=1).min().shift(1)
        result[f'{variable}_pct_diff_meanlast12_meanhist'] = 1 - (result[f'{variable}_mean_past_year'] / result[f'{variable}_past_historic_mean'])
        return result
    
    # Apply the feature calculations to the DataFrame
    aggr_data = pd.DataFrame()
    for basin_id in exog_series_df.columns[:-3]:  # Exclude datetime-related columns
        basin_data = exog_series_df[[basin_id]].copy()  # Data for a specific basin
        basin_features = calculate_features(basin_data)
        basin_features['basin_id'] = basin_id
        basin_features['Year'] = exog_series_df['year']
        basin_features['Month'] = exog_series_df['month']
        aggr_data = pd.concat([aggr_data, basin_features])
    
    # Merge the aggregated data with the global training DataFrame
    global basin_training_data_df
    basin_training_data_df = basin_training_data_df.merge(aggr_data, on=['Year', 'Month', 'basin_id'], how='left')
    
    # 9. Clean up memory and delete the file
    del exog_var, exog_series_df, aggr_data, basin_time_series, basin_time_series_df
    gc.collect()
    
    # Remove the file if needed
    if os.path.exists(filename):
        os.remove(filename)

    # Save the updated training data
    basin_training_data_df.to_csv('temp_exog_basin_training_data_df.csv', index= False)

# Load basin information
basin_info = gpd.read_file('basin_info_df.csv')
basin_info['geometry'] = basin_info['geometry'].apply(wkt.loads)

# Create a GeoDataFrame from the basin information
gdf = gpd.GeoDataFrame(basin_info, geometry='geometry')
gdf.set_crs("EPSG:4326", inplace=True)

# Load training data and update column names
basin_training_data_df = pd.read_csv('basin_training_data_df.csv')
basin_training_data_df.rename(columns={'id_loc': 'basin_id'}, inplace=True)
basin_training_data_df.basin_id = basin_training_data_df.basin_id.astype('str')

# Loop through the list of variables and process each one
for var in variables:
    download_and_process(var)

print("Process completed successfully.")

Archivo download_100m_u_component_of_wind.nc ya existe, saltando descarga.
Usando la variable 'u100' del archivo download_100m_u_component_of_wind.nc
Descargando download_100m_v_component_of_wind.nc...


2024-09-22 23:24:52,112 INFO Request ID is 50e0d03a-aac5-4482-ab50-66e51b3db73e
2024-09-22 23:24:52,217 INFO status has been updated to accepted
2024-09-22 23:24:53,800 INFO status has been updated to running
2024-09-22 23:24:56,097 INFO status has been updated to successful


Usando la variable 'v100' del archivo download_100m_v_component_of_wind.nc
Descargando download_10m_u_component_of_wind.nc...


2024-09-22 23:27:58,111 INFO Request ID is bc8f7306-9a02-4596-92ca-26e1a6f49a11
2024-09-22 23:27:58,179 INFO status has been updated to accepted
2024-09-22 23:27:59,737 INFO status has been updated to running
2024-09-22 23:29:12,580 INFO status has been updated to successful


Usando la variable 'u10' del archivo download_10m_u_component_of_wind.nc
Descargando download_10m_v_component_of_wind.nc...


...

Usando la variable 'p91.162' del archivo download_vertical_integral_of_northward_cloud_frozen_water_flux.nc
Descargando download_vertical_integral_of_eastward_cloud_frozen_water_flux.nc...


2024-09-23 03:12:39,500 INFO Request ID is e8ff9349-c312-49fc-bfb0-7fda638dee42
2024-09-23 03:12:39,734 INFO status has been updated to accepted
2024-09-23 03:12:41,297 INFO status has been updated to running
2024-09-23 03:13:54,090 INFO status has been updated to successful


Usando la variable 'p90.162' del archivo download_vertical_integral_of_eastward_cloud_frozen_water_flux.nc
Descargando download_vertical_integral_of_northward_cloud_liquid_water_flux.nc...


2024-09-23 03:16:27,362 INFO Request ID is b2f54068-c19a-4eef-832a-28beb772840d
2024-09-23 03:16:27,424 INFO status has been updated to accepted
2024-09-23 03:16:28,987 INFO status has been updated to running
2024-09-23 03:17:41,843 INFO status has been updated to successful


Usando la variable 'p89.162' del archivo download_vertical_integral_of_northward_cloud_liquid_water_flux.nc
Proceso completado con éxito.


### Checking the results

In [26]:
basin_training_data_df.head()

,lag_60,lag_59,lag_58,lag_57,lag_56,lag_55,lag_54,lag_53,lag_52,lag_51,lag_50,lag_49,lag_48,lag_47,lag_46,lag_45,lag_44,lag_43,lag_42,lag_41,lag_40,lag_39,lag_38,lag_37,lag_36,lag_35,lag_34,lag_33,lag_32,lag_31,lag_30,lag_29,lag_28,lag_27,lag_26,lag_25,lag_24,lag_23,lag_22,lag_21,lag_20,lag_19,lag_18,lag_17,lag_16,lag_15,lag_14,lag_13,lag_12,lag_11,lag_10,lag_9,lag_8,lag_7,lag_6,lag_5,lag_4,lag_3,lag_2,lag_1,step_1,step_2,step_3,step_4,step_5,step_6,step_7,step_8,step_9,step_10,step_11,step_12,step_13,step_14,step_15,step_16,step_17,step_18,step_19,step_20,step_21,step_22,step_23,step_24,step_25,step_26,step_27,step_28,step_29,step_30,step_31,step_32,step_33,step_34,step_35,step_36,step_37,step_38,step_39,step_40,step_41,step_42,step_43,step_44,step_45,step_46,step_47,step_48,step_49,step_50,step_51,step_52,step_53,step_54,step_55,step_56,step_57,step_58,step_59,step_60,basin_id,hist_mean,hist_max,hist_min,hist_std,hist_vc,mean_past_year,max_past_year,min_past_year,mean_past_quarter,max_past_quarter,min_past_quarter,pct_diff_meanlast12_meanhist,otm_hist_0,otm_hist_1,otm_hist_2,otm_hist_3,otm_hist_4,otm_hist_5,otm_hist_6,otm_hist_7,otm_hist_8,otm_hist_9,otm_hist_10,otm_hist_11,Year,Month,Month_sin,is_Winter,is_Spring,is_Summer,is_Autumn,100m_u_component_of_wind_past_historic_mean,100m_u_component_of_wind_past_historic_max,100m_u_component_of_wind_past_historic_min,100m_u_component_of_wind_mean_past_year,100m_u_component_of_wind_max_past_year,100m_u_component_of_wind_min_past_year,100m_u_component_of_wind_mean_past_quarter,100m_u_component_of_wind_max_past_quarter,100m_u_component_of_wind_min_past_quarter,100m_u_component_of_wind_mean_past_5years,100m_u_component_of_wind_max_past_5years,100m_u_component_of_wind_min_past_5years,100m_u_component_of_wind_pct_diff_meanlast12_meanhist,100m_v_component_of_wind_past_historic_mean,100m_v_component_of_wind_past_historic_max,100m_v_component_of_wind_past_historic_min,100m_v_component_of_wind_mean_past_year,100m_v_component_of_wind_max_past_year,100m_v_component_of_wind_min_past_year,100m_v_component_of_wind_mean_past_quarter,100m_v_component_of_wind_max_past_quarter,100m_v_component_of_wind_min_past_quarter,100m_v_component_of_wind_mean_past_5years,100m_v_component_of_wind_max_past_5years,100m_v_component_of_wind_min_past_5years,100m_v_component_of_wind_pct_diff_meanlast12_meanhist,10m_u_component_of_wind_past_historic_mean,10m_u_component_of_wind_past_historic_max,10m_u_component_of_wind_past_historic_min,10m_u_component_of_wind_mean_past_year,10m_u_component_of_wind_max_past_year,10m_u_component_of_wind_min_past_year,10m_u_component_of_wind_mean_past_quarter,10m_u_component_of_wind_max_past_quarter,10m_u_component_of_wind_min_past_quarter,10m_u_component_of_wind_mean_past_5years,10m_u_component_of_wind_max_past_5years,10m_u_component_of_wind_min_past_5years,10m_u_component_of_wind_pct_diff_meanlast12_meanhist,10m_v_component_of_wind_past_historic_mean,10m_v_component_of_wind_past_historic_max,10m_v_component_of_wind_past_historic_min,10m_v_component_of_wind_mean_past_year,10m_v_component_of_wind_max_past_year,10m_v_component_of_wind_min_past_year,10m_v_component_of_wind_mean_past_quarter,10m_v_component_of_wind_max_past_quarter,10m_v_component_of_wind_min_past_quarter,10m_v_component_of_wind_mean_past_5years,10m_v_component_of_wind_max_past_5years,10m_v_component_of_wind_min_past_5years,10m_v_component_of_wind_pct_diff_meanlast12_meanhist,10m_wind_speed_past_historic_mean,10m_wind_speed_past_historic_max,10m_wind_speed_past_historic_min,10m_wind_speed_mean_past_year,10m_wind_speed_max_past_year,10m_wind_speed_min_past_year,10m_wind_speed_mean_past_quarter,10m_wind_speed_max_past_quarter,10m_wind_speed_min_past_quarter,10m_wind_speed_mean_past_5years,10m_wind_speed_max_past_5years,10m_wind_speed_min_past_5years,10m_wind_speed_pct_diff_meanlast12_meanhist,2m_dewpoint_temperature_past_historic_mean,2m_dewpoint_temperature_past_historic_max,2m_dewpoint_temperature_past_historic_min,2m_dewpo

### Saving the resulting dataset

In [27]:
basin_training_data_df.to_csv('basin_training_data_df_with_exog.csv', index = False)